# ENCODER DECODER MODEL - Google's T5 base

In [1]:
# Import libraries
!pip install datasets
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import json
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import json

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Load the dataset
with open('/content/qa_data final.json', 'r') as f:
    faq_data = json.load(f)
with open('/content/test_data_100.json', 'r') as f:
    test_data = json.load(f)


# Prepare the dataset
questions = [item['question'] for item in faq_data]
answers = [item['answer'] for item in faq_data]
# Prepare the dataset
questions_t = [item['question'] for item in test_data]
answers_t = [item['answer'] for item in test_data]

# Create a Hugging Face Dataset
data = {'question': questions, 'answer': answers}
dataset = Dataset.from_dict(data)
# Create a Hugging Face test Dataset
data = {'question': questions_t, 'answer': answers_t}
test_dataset = Dataset.from_dict(data)

# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')
model.to(device)

# Preprocess the data
def preprocess_function(examples):
    inputs = ['answer_question: ' + q for q in examples['question']]
    model_inputs = tokenizer(
        inputs,
        padding='max_length',  # Dynamically pad
        truncation=True,
        max_length=256,
        return_tensors='pt'  # Ensure it's in PyTorch format
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['answer'],
            padding='max_length',  # Dynamically pad
            truncation=True,
            max_length=256,
            return_tensors='pt'  # Ensure it's in PyTorch format
        )
    # Replace padding token id's of the labels by -100 so it's ignored by the loss
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels_example]
        for labels_example in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the dataset
dataset = dataset.map(preprocess_function, batched=True)
# Apply preprocessing to the test dataset
test_dataset = dataset.map(preprocess_function, batched=True)

# Split dataset into training and validation sets
train_dataset = dataset
val_dataset =test_dataset

# Training arguments
training_args = TrainingArguments(
    output_dir='/content/results',  # Output directory
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=5e-5,  # Lower learning rate
    per_device_train_batch_size=8,  # Adjust batch size based on your GPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Tune based on the dataset size
    weight_decay=0.01,  # Prevent overfitting
    logging_dir='/content/logs',  # Log directory
    # save_steps=500,  # Remove this line to disable saving by steps
    save_strategy="epoch", # Save at the end of each epoch
    warmup_steps=500,  # Learning rate warm-up
    logging_steps=100,  # Log every 100 steps
    load_best_model_at_end=True,  # Load the best model at the end
    metric_for_best_model="loss",  # You can change this to another metric like accuracy
    gradient_accumulation_steps=1,  # Accumulate gradients if necessary
    gradient_checkpointing=True,  # Use gradient checkpointing for memory efficiency
)
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=920, training_loss=2.531030630028766, metrics={'train_runtime': 1993.2101, 'train_samples_per_second': 3.672, 'train_steps_per_second': 0.462, 'total_flos': 2228785879449600.0, 'train_loss': 2.531030630028766, 'epoch': 10.0})

# Conversation Loop:

In [5]:
# Function to generate an answer using the T5 model
def generate_answer(question):
    input_text = 'answer_question: ' + question
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=256, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=256,
            num_beams=5,
            early_stopping=True,
            no_repeat_ngram_size=2,
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Chatbot loop
print("Welcome to the University FAQ Chatbot! Type 'exit' to end the conversation.\n")
while True:
    user_query = input("You: ")
    if user_query.lower() == 'exit':
        print("Chatbot: Thank you for using the FAQ Chatbot. Goodbye!")
        break
    response = generate_answer(user_query)
    print(f"Chatbot: {response}\n")

Welcome to the University FAQ Chatbot! Type 'exit' to end the conversation.

You: where is registrar office located
Chatbot: The registrar’s office is located on the ground floor of the building.

You: where to go if i am mentally down
Chatbot: If you are mentally down, you can go to the mental health center.

You: what does office of academic performance do
Chatbot: The office of academic performance monitors student performance and ensures that it meets the academic standards set by the university.

You: how many minors does habib offer
Chatbot: Habib University offers a total of 16 minors, including four majors and one minor.

You: exit
Chatbot: Thank you for using the FAQ Chatbot. Goodbye!


# Evaluation:

In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [4]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1e58e0f02ff59486017595a24d10c98ceeffcff3eefe3cf3a7a8a4a2a42fc05b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [6]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset
from evaluate import load
import json

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Load the fine-tuned model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('/content/results/checkpoint-920')
model = T5ForConditionalGeneration.from_pretrained('/content/results/checkpoint-920')
model.to(device)

# Load the test dataset (assuming it's in test.json format)
with open('/content/test_data_100.json', 'r') as f:
    test_data = json.load(f)

# Prepare the dataset
questions = [item['question'] for item in test_data]
answers = [item['answer'] for item in test_data]

# Create a Hugging Face Dataset
data = {'question': questions, 'answer': answers}
test_dataset = Dataset.from_dict(data)

# Load evaluation metrics
rouge = load("rouge")
bleu = load("bleu")

# Generate predictions
def generate_predictions(example):
    # Preprocess input
    inputs = tokenizer("answer_question: " + example['question'], return_tensors="pt", max_length=256, truncation=True, padding='max_length').to(device)
    # Generate answer
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=256)
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {'predicted_answer': predicted_answer}

# Apply the prediction function to the test dataset
predictions = test_dataset.map(generate_predictions, batched=False)

# Calculate ROUGE and BLEU scores
references = predictions['answer']
generated = predictions['predicted_answer']

# ROUGE Evaluation
rouge_score = rouge.compute(predictions=generated, references=references)
print("ROUGE Score:", rouge_score)

# BLEU Evaluation
bleu_score = bleu.compute(predictions=generated, references=references)
print("BLEU Score:", bleu_score)


Using device: cuda


Map:   0%|          | 0/84 [00:00<?, ? examples/s]

ROUGE Score: {'rouge1': 0.29944061432046565, 'rouge2': 0.10944683343986294, 'rougeL': 0.24859204293425308, 'rougeLsum': 0.24894174305468347}
BLEU Score: {'bleu': 0.08170688242100423, 'precisions': [0.33754681647940077, 0.11793372319688109, 0.0625, 0.034501061571125265], 'brevity_penalty': 0.8488628252211823, 'length_ratio': 0.8592115848753017, 'translation_length': 2136, 'reference_length': 2486}
